In [1]:
import pandas as pd
import numpy as np

from my_functions_idealisedprofiles import *

### Get version of landcover array with just 'urban' and 'rural' categories

In [2]:
# Read in the data
landcover, out_meta = prepare_rainfall_scenario_raster(model_directory + "LandCover_clipped.tif", True)
# Convert the 1 and 6 values to 10 (for urban) and the rest to 11 (for non-urban).  
landcover_mod =  np.where(landcover==1, 10, landcover)
landcover_mod =  np.where(landcover_mod==6, 10, landcover_mod)
# Convert the rest of the classes to 11
for i in [1,2,3,4,5,7,8,9]:
    landcover_mod =  np.where(landcover_mod==i, 11, landcover_mod)

### Define the names of the method (shorter and longer versions)

In [70]:
methods =['6h_feh_sp', '6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5',
          '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9']
short_ids = ['6h_feh_sp', '6h_sp_fl_0.1', '6h_sp_fl_0.2', '6h_sp_fl_0.3', '6h_sp_fl_0.4', '6h_sp_c_0.5',
          '6h_sp_bl_0.6','6h_sp_bl_0.7','6h_sp_bl_0.8','6h_sp_bl_0.9']   

### Find maximum intensity for each method and minute in which it occurs (to use in sorting results analysis)

In [71]:
maxs = []
min_of_maxs = []

# Add FEH data
feh_precip=pd.read_csv("../../CreateSyntheticRainfallEvents/ReFH2_singlepeak/6hr_100yrRP/PostLossRemoval/6h_feh_singlepeak_urban.csv")
maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
min_of_maxs.append(feh_precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

for method in methods[1:]:
#     if method == '6h_single-peak':
#         precip=pd.read_csv("../../CreateSyntheticRainfallEvents/SyntheticProfiles/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
#     else:
    precip=pd.read_csv("../../CreateSyntheticRainfallEvents/IdealisedProfiles/6hr_100yrRP/PostLossRemoval/{}_urban.csv".format(method))
    maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].max())
    min_of_maxs.append(precip["Total net rain mm (Observed rainfall - 01/08/2022) - urbanised model"].idxmax())

### Create versions of lists of methods, in order based on max intensity and the the timing of the max intensity 

In [72]:
short_ids_by_loading=  pd.DataFrame({"min": min_of_maxs, 'method_name': short_ids}).sort_values('min')["method_name"].tolist()
short_ids_by_loading.remove('6h_feh_sp')
short_ids_by_loading = ['6h_feh_sp']+short_ids_by_loading

short_ids_by_intensity = pd.DataFrame({"min": maxs, 'method_name': short_ids}).sort_values('min', ascending = False)["method_name"].tolist()
short_ids_by_intensity.remove('6h_feh_sp')
short_ids_by_intensity = ['6h_feh_sp']+short_ids_by_intensity

### Create dataframe of colours for each cluster (based on their loading)

In [73]:
colours = ['black'] + ['darkblue']*2 + ['paleturquoise']*2 + ['grey']+  ['indianred']*2 + ['darkred']*2
loadings = ['FEH'] + ['F2']*2 + ['F1']*2 + ['C']+  ['B1']*2 + ['B2']*2
colours_df = pd.DataFrame({ 'short_id': short_ids_by_loading, "colour": colours, 'loading':loadings})
colours_df = colours_df.reindex(colours_df['short_id'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
colours_df.reset_index(inplace=True, drop=True)

In [74]:
def create_colours_df (short_ids_by_loading, short_ids):
    colours = ['black'] + ['darkblue']*2 + ['paleturquoise']*2 + ['grey']+  ['indianred']*2 + ['darkred']*2
    loadings = ['FEH'] + ['F2']*2 + ['F1']*2 + ['C']+  ['B1']*2 + ['B2']*2
    colours_df = pd.DataFrame({ 'short_id': short_ids_by_loading, "colour": colours, 'loading':loadings})
    colours_df = colours_df.reindex(colours_df['short_id'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
    colours_df.reset_index(inplace=True, drop=True)
    return colours_df

In [75]:
colours_df = create_colours_df(short_ids_by_loading, short_ids)

### Create list of filepaths, formatted to be used for either depth or velocity

In [76]:
fps = []
for method_num, short_id in enumerate(short_ids):
    fp = model_directory + "{}/{} (Max).Resampled.Terrain.tif".format(short_id, '{}')
    fps.append(fp)

### Define breaks for categorising velocity and depth

In [77]:
# Define breaks to split the depths/velocities on
breaks_depths = np.array([0, 0.3, 0.6, 1.2, 100])  
labels_depth = ['<=0.3m', '0.3-0.6m', '0.6-1.2m', '>1.2m']
breaks_velocity = np.array([0,0.25,0.5,2,100])
labels_velocity = ["<=0.25m/s", "0.25-0.5m/s", "0.5-2m/s", ">2m/s"]

# <u> Flood extent </u>
To examine whether the rainfall's temporal distribution influences the total extent of flooding, the number of flooded cells and the total flooded area in km2 (incl. only cells with depth >0.1m) is compared between the profile with a single peak, and the three methods for producing multi-peaked rainfall events. b

### Create dataframes containing the (total/urban) flooded area in each depth/velocity bin

In [78]:
def create_binned_counts_and_props(fps, variable_name, breaks, labels, remove_little_values):
    # Create dataframes to populate with values
    counts_df = pd.DataFrame()
    proportions_df = pd.DataFrame()        

    # Loop through each rainfall scenario
    # Get the raster containing its values, and count the number of each unique value, and construct into a dataframe
    for fp in fps  :
        # Classify depth/velocity rasters into depth/velocity bins
        raster = prepare_rainfall_scenario_raster(fp.format(variable_name), remove_little_values)[0]
        unique, counts = np.unique(raster, return_counts=True)
        df = pd.DataFrame({'values': unique, 'counts':counts})

        # Add a new column specifying the bin which each value falls within
        df['bins']= pd.cut(unique, bins=breaks, right=False)

        # Create a new dataframe showing the number of cells in each of the bins
        groups = df.groupby(['bins']).sum()
        groups  = groups.reset_index()

        # Find the total number of cells
        total_n_cells = groups ['counts'].sum()
        # Find the number of cells in each group as a proportion of the total
        groups['Proportion'] = round((groups['counts']/total_n_cells) *100,1)

        # Add values to dataframes
        method_name = re.search('{}(.*)/'.format(model_directory), fp).group(1)
        counts_df[method_name] = groups['counts']
        proportions_df[method_name] = groups['Proportion']

    # Reset index to show the groups
    counts_df.reset_index(inplace=True)
    proportions_df.reset_index(inplace=True)

    # Set index values
    counts_df['index'] = labels
    proportions_df['index'] = labels

    return counts_df, proportions_df

In [81]:
def create_binned_counts_and_props(fps, variable_name, breaks, labels, remove_little_values):
    # Create dataframes to populate with values
    counts_df = pd.DataFrame()
    proportions_df = pd.DataFrame()        

    # Loop through each rainfall scenario
    # Get the raster containing its values, and count the number of each unique value, and construct into a dataframe
    for fp in fps  :
        if fp == '../../../../FloodModelling/MeganModel_New/6h_sp_c_0.5/{} (Max).Resampled.Terrain.tif':
            fp = '../../../../FloodModelling/MeganModel_New//6h_sp/(Max).Resampled.Terrain.tif'
        else:   
            # Classify depth/velocity rasters into depth/velocity bins
            raster = prepare_rainfall_scenario_raster(fp.format(variable_name), remove_little_values)[0]
            unique, counts = np.unique(raster, return_counts=True)
            df = pd.DataFrame({'values': unique, 'counts':counts})

            # Add a new column specifying the bin which each value falls within
            df['bins']= pd.cut(unique, bins=breaks, right=False)

            # Create a new dataframe showing the number of cells in each of the bins
            groups = df.groupby(['bins']).sum()
            groups  = groups.reset_index()

            # Find the total number of cells
            total_n_cells = groups ['counts'].sum()
            # Find the number of cells in each group as a proportion of the total
            groups['Proportion'] = round((groups['counts']/total_n_cells) *100,1)

            # Add values to dataframes
            method_name = re.search('{}(.*)/'.format(model_directory), fp).group(1)
            counts_df[method_name] = groups['counts']
            proportions_df[method_name] = groups['Proportion']

    # Reset index to show the groups
    counts_df.reset_index(inplace=True)
    proportions_df.reset_index(inplace=True)

    # Set index values
    counts_df['index'] = labels
    proportions_df['index'] = labels

    return counts_df, proportions_df

In [82]:
velocity_counts, velocity_props = create_binned_counts_and_props(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values)
depth_counts, depth_props = create_binned_counts_and_props(fps, 'Depth', breaks_depths, labels_depth, remove_little_values)

velocity_counts_urban, velocity_props_urban = create_binned_counts_and_props_urban(fps, 'Velocity', breaks_velocity, labels_velocity, remove_little_values, landcover_mod)
depth_counts_urban, depth_props_urban = create_binned_counts_and_props_urban(fps, 'Depth', breaks_depths, labels_depth, remove_little_values, landcover_mod)

RasterioIOError: ../../../../FloodModelling/MeganModel_New/6h_sp_c_0.5/Velocity (Max).Resampled.Terrain.tif: No such file or directory

### Create dataframes containing the (total/urban) flooded area

In [ ]:
totals_df = create_totals_df(velocity_counts)
totals_df_urban = create_totals_df(velocity_counts_urban)      

### Create dataframes containing the % diff in the flooded area between single peak and each other method  

In [ ]:
percent_diffs_df = find_percentage_diff (totals_df, fps) 
percent_diffs_df_urban = find_percentage_diff (totals_df_urban, fps)   

## Find number of cells in which each method leads to the worst flooding (depth/velocity)

In [ ]:
# Find the number of flooded cells with the worst flooding for each method
worst_case_method_depth = find_worst_case_method(fps, short_ids, 'Depth')
worst_case_method_velocity = find_worst_case_method(fps, short_ids,  'Velocity') 

In [ ]:
# Remove multiple matches and nan
worst_case_method_depth = worst_case_method_depth[~worst_case_method_depth['values'].isin(['multiple matches','nan'])]
worst_case_method_velocity = worst_case_method_velocity[~worst_case_method_velocity['values'].isin(['multiple matches','nan'])]

# # Reorder (and also add in the methods that are missing)
worst_case_method_depth = pd.merge(worst_case_method_depth,  pd.DataFrame({'values': short_ids}), how="outer")
worst_case_method_depth = worst_case_method_depth.reindex(worst_case_method_depth['values'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
worst_case_method_depth.reset_index(inplace=True,drop=True)

worst_case_method_velocity = pd.merge(worst_case_method_velocity,  pd.DataFrame({'values': short_ids}), how="outer")
worst_case_method_velocity = worst_case_method_velocity.reindex(worst_case_method_velocity['values'].map(dict(zip(short_ids, range(len(short_ids))))).sort_values().index)
worst_case_method_velocity.reset_index(inplace=True,drop=True)

## Find number of cells with each hazard rating

In [25]:
def create_binned_counts_and_props_hazard(fps):

    # Create dataframes to populate with values
    counts_df = pd.DataFrame()
    proportions_df = pd.DataFrame()      

    for fp in fps:
        # Define filepath
        fp = '../../../../FloodModelling/MeganModel_New/{}/hazard_classified.tif'.format(fp.split('New/')[1].split('/{}')[0])
        # Read in data
        hazard = prepare_rainfall_scenario_raster(fp, remove_little_values)[0]
        # Count the number of each value
        unique, counts = np.unique(hazard, return_counts=True)
        df = pd.DataFrame({'values': unique, 'counts':counts})
        # Remove Nan values
        df = df.dropna()

        # Find the total number of cells
        total_n_cells = df ['counts'].sum()
        # Find the number of cells in each group as a proportion of the total
        df['Proportion'] = round((df['counts']/total_n_cells) *100,1)
        
        # Add values to dataframes
        method_name = re.search('{}(.*)/'.format(model_directory), fp).group(1)
        counts_df[method_name] = df['counts']
        proportions_df[method_name] = df['Proportion']

    # Reset index to show the groups
    counts_df.reset_index(inplace=True)
    proportions_df.reset_index(inplace=True)

    # Set index values
    labels_hazard = ['Low hazard', 'Moderate hazard', 'Significant hazard', 'Extreme hazard']
    counts_df['index'] = labels_hazard
    proportions_df['index'] = labels_hazard
    return counts_df, proportions_df

In [26]:
hazard_counts, hazard_props = create_binned_counts_and_props_hazard(fps)

## Find number of cells which have moved between hazard categories

In [19]:
def create_binned_counts_and_props_hazard_cat_change(fps):
    # Create dataframes to populate with values
    counts_df = pd.DataFrame(columns = ["values"])
    proportions_df = pd.DataFrame(columns = ["values"]) 

    for fp in fps[1:]:
        # Add values to dataframes
        method_name = re.search('{}(.*)/'.format(model_directory), fp).group(1)
        # Read in hazard data 
        fp = '../../../../FloodModelling/MeganModel_New/{}/hazard_cat_difference.tif'.format(fp.split('New/')[1].split('/{}')[0])
        hazard = prepare_rainfall_scenario_raster(fp, False)[0]
        unique, counts = np.unique(hazard, return_counts=True)
        df = pd.DataFrame({'values': unique, method_name:counts})
        # Remove NA columns
        df = df.dropna()

        # Add to dataframes
        counts_df= counts_df.merge(df[['values', method_name]], on = 'values', how = 'outer')

        # Find the total number of cells
        total_n_cells = df [method_name].sum()
        # Find the number of cells in each group as a proportion of the total
        df[method_name] = round((df[method_name]/total_n_cells) *100,1)

       # Add to dataframes
        proportions_df= proportions_df.merge(df[['values', method_name]], on = 'values', how = 'outer')

        # Order intoi ascending order
        proportions_df = proportions_df.sort_values(by='values')
        counts_df = counts_df.sort_values(by='values')

    # Join the two dataframes together and reformat
    both_dfs = pd.DataFrame(columns = ["Cluster_num"])  
    for num, df in enumerate([counts_df,proportions_df]):
        # Some categories might be missing which causes this to throw an error
        df['Cluster_num']= ['Hazard_2CatsLower', 'Hazard_1CatsLower', 'Hazard_SameCat', 'Hazard_1CatsHigher', 'Hazard_2CatsHigher', 'Hazard_3CatsHigher']
        del df['values']
        df = df.set_index('Cluster_num').T
        if num == 0:
            df = df.add_suffix('_countcells')
        else:
            df = df.add_suffix('_propcells')
        df['Cluster_num'] = df.index
        both_dfs = pd.merge(both_dfs, df,  how="outer", on = 'Cluster_num')
    
    return both_dfs


In [27]:
hazard_cat_changes = create_binned_counts_and_props_hazard_cat_change(fps)

### Create a dataframe containing all the info on each of the scenarios

In [29]:
cluster_results = pd.DataFrame({'Cluster_num': short_ids, "MaxRainfallIntensity": maxs,  
    "MaxRainfallIntensityMinute": min_of_maxs,
   'TotalFloodedArea':totals_df['FloodedArea'],'%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs'],
    '%Diff_FloodedArea_fromSP_formatted':percent_diffs_df['percent_diff_formatted'],
    'Abs%Diff_FloodedArea_fromSP':percent_diffs_df['percent_diffs_abs'],'UrbanFloodedArea':totals_df_urban['FloodedArea'],
  '%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs'] ,
  '%Diff_UrbanFloodedArea_fromSP_formatted':percent_diffs_df_urban['percent_diff_formatted'],
    'Abs%Diff_UrbanFloodedArea_fromSP':percent_diffs_df_urban['percent_diffs_abs'], 
    'WorstCaseDepth_ncells': worst_case_method_depth['counts'].tolist(),
    'WorstCaseVelocity_ncells': worst_case_method_velocity['counts'].tolist(), 'colour':colours_df['colour']}) 

### Add the depth/velocity category breakdowns and hazard categories to this

In [30]:
dfs = [velocity_props, depth_props, velocity_props_urban, depth_props_urban, velocity_counts, depth_counts,
          velocity_counts_urban, depth_counts_urban,hazard_counts, hazard_props]
suffixes = ['_propcells', '_propcells','_propcells_urban','_propcells_urban','_countcells','_countcells','_countcells_urban','_countcells_urban',
'_numcells', '_propcells']

for num, df in enumerate(dfs):
    # Reformat the dataframe
    df = df.set_index('index').T
    # Add the correct suffix to the column names
    df = df.add_suffix(suffixes[num]) 
    # Add Cluster_num column for joining
    df['Cluster_num'] = df.index#
    # Join to cluster results dataframe
    cluster_results = pd.merge(cluster_results,  df, how="outer", on = 'Cluster_num')
    
cluster_results = pd.merge(cluster_results, hazard_cat_changes,  how="outer", on = 'Cluster_num')    

### Save to file

In [31]:
cluster_results.to_csv("Data/allclusters_summary.csv", index=False)

In [32]:
cluster_results

,Cluster_num,MaxRainfallIntensity,MaxRainfallIntensityMinute,TotalFloodedArea,%Diff_FloodedArea_fromSP,%Diff_FloodedArea_fromSP_formatted,Abs%Diff_FloodedArea_fromSP,UrbanFloodedArea,%Diff_UrbanFloodedArea_fromSP,%Diff_UrbanFloodedArea_fromSP_formatted,...,Hazard_SameCat_countcells,Hazard_1CatsHigher_countcells,Hazard_2CatsHigher_countcells,Hazard_3CatsHigher_countcells,Hazard_2CatsLower_propcells,Hazard_1CatsLower_propcells,Hazard_SameCat_propcells,Hazard_1CatsHigher_propcells,Hazard_2CatsHigher_propcells,Hazard_3CatsHigher_propcells
0,6h_feh_sp,0.201255,180,1.701144,0.00,,0.00,0.575595,0.00,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6h_sp_fl_0.1,0.232031,36,1.636911,-3.78,-3.78%,3.78,0.540120,-6.16,-6.16%,...,1519965.0,61215.0,3774.0,3.0,0.0,1.9,94.0,3.8,0.2,0.0
2,6h_sp_fl_0.2,0.243904,72,1.635671,-3.85,-3.85%,3.85,0.514048,-10.69,-10.69%,...,1500827.0,58382.0,3824.0,5.0,0.0,2.9,93.2,3.6,0.2,0.0
3,6h_sp_fl_0.3,0.254309,108,1.654627,-2.73,-2.73%,2.73,0.523471,-9.06,-9.06%,...,1526429.0,47659.0,2657.0,3.0,0.0,3.0,93.8,2.9,0.2,0.0
4,6h_sp_fl_0.4,0.262907,144,1.678387,-1.34,-1.34%,1.34,0.532838,-7.43,-7.43%,...,1548240.0,37617.0,1750.0,3.0,0.1,3.2,94.3,2.3,0.1,0.0
5,6h_sp_bl_0.6,0.292262,215,1.729529,1.67,+1.67%,1.67,0.551026,-4.27,-4.27%,...,1568533.0,24623.0,761.0,3.0,0.2,4.0,94.3,1.5,0.0,0.0
6,6h_sp_bl_0.7,0.317888,251,1.746964,2.69,+2.69%,2.69,0.559568,-2.78,-2.78%,...,1564776.0,25369.0,503.0,3.0,0.3,4.4,93.8,1.5,0.0,0.0
7,6h_sp_bl_0.8,0.343370,287,1.764415,3.72,+3.72%,3.72,0.567784,-1.36,-1.36%,...,1549395.0,33993.0,394.0,2.0,0.4,5.0,92.6,2.0,0.0,0.0
8,6h_sp_bl_0.9,0.368805,323,1.775070,4.35,+4.35%,4.35,0.574021,-0.27,-0.27%,...,1522536.0,53425.0,317.0,2.0,0.4,5.4,91.0,3.2,0.0,0.0


### Delete tiff files (as these aren't used again and take up a lot of space)

In [ ]:
for method in short_ids:
    print(method)
    if method != '6h_feh_sp':
        os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_cat_difference.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_classified.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_difffromsinglepeak_posneg.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_classified.tif".format(method)) 
        os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_difffromsinglepeak_posneg.tif".format(method)) 
        
    os.remove("../../../../FloodModelling/MeganModel_New/{}/Depth_classified.tif".format(method)) 
    os.remove("../../../../FloodModelling/MeganModel_New/{}/hazard_classified.tif".format(method)) 
    os.remove("../../../../FloodModelling/MeganModel_New/{}/Velocity_classified.tif".format(method)) 